In [ ]:
import pandas as pd
import warnings
import requests, zipfile, io, os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
warnings.filterwarnings('ignore')

In [ ]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [ ]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [ ]:
download_and_extract(healthy_url, healthy_path)

In [ ]:
download_and_extract(parkinson_url, parkinson_path)

In [ ]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'

In [ ]:
# define Input and labels (0 = healthy, parkinson = 1)
X = []
y = []
seq = 500

In [ ]:
# create dataset with sequence length 500

# healthy signals
for filename in os.listdir(healthy_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(0)

# parkinson signals
for filename in os.listdir(parkinson_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(1) 


In [ ]:
len(X)

29088

In [ ]:
# convert lists to np arrays
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

(29088, 500, 6)

In [ ]:
y.shape

(29088,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_valid = scaler.transform(X_valid.reshape(-1, X_valid.shape[-1])).reshape(X_valid.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [ ]:
model = keras.models.Sequential([
keras.layers.GRU(128, return_sequences=True,  input_shape=(X_train.shape[1], 6)),
keras.layers.GRU(128),
keras.layers.Dense(1, activation="sigmoid")
])


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
checkpoint_filepath = "model.h5"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.fit(X_train, y_train, epochs = 30, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/30
582/582 [==============================] - 103s 176ms/step - loss: 0.6103 - accuracy: 0.6681 - val_loss: 0.5930 - val_accuracy: 0.6859
Epoch 2/30
582/582 [==============================] - 105s 180ms/step - loss: 0.5788 - accuracy: 0.6912 - val_loss: 0.5547 - val_accuracy: 0.7041
Epoch 3/30
582/582 [==============================] - 107s 184ms/step - loss: 0.5352 - accuracy: 0.7288 - val_loss: 0.5249 - val_accuracy: 0.7480
Epoch 4/30
582/582 [==============================] - 107s 184ms/step - loss: 0.4796 - accuracy: 0.7704 - val_loss: 0.4613 - val_accuracy: 0.7727
Epoch 5/30
582/582 [==============================] - 106s 182ms/step - loss: 0.4431 - accuracy: 0.7945 - val_loss: 0.4827 - val_accuracy: 0.7664
Epoch 6/30
582/582 [==============================] - 103s 177ms/step - loss: 0.4042 - accuracy: 0.8117 - val_loss: 0.4097 - val_accuracy: 0.8017
Epoch 7/30
582/582 [==============================] - 102s 176ms/step - loss: 0.3596 - accuracy: 0.8409 - val_loss: 0.3951 -

In [ ]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/20
582/582 [==============================] - 105s 180ms/step - loss: 0.0699 - accuracy: 0.9747 - val_loss: 0.2885 - val_accuracy: 0.9031
Epoch 2/20
582/582 [==============================] - 105s 180ms/step - loss: 0.0699 - accuracy: 0.9750 - val_loss: 0.2942 - val_accuracy: 0.9042
Epoch 3/20
582/582 [==============================] - 103s 178ms/step - loss: 0.0657 - accuracy: 0.9755 - val_loss: 0.2949 - val_accuracy: 0.9050
Epoch 4/20
582/582 [==============================] - 106s 182ms/step - loss: 0.0590 - accuracy: 0.9783 - val_loss: 0.2872 - val_accuracy: 0.9091
Epoch 5/20
582/582 [==============================] - 108s 185ms/step - loss: 0.0525 - accuracy: 0.9811 - val_loss: 0.3001 - val_accuracy: 0.9132
Epoch 6/20
582/582 [==============================] - 106s 182ms/step - loss: 0.0465 - accuracy: 0.9825 - val_loss: 0.2921 - val_accuracy: 0.9108
Epoch 7/20
582/582 [==============================] - 102s 175ms/step - loss: 0.0651 - accuracy: 0.9752 - val_loss: 0.2916 -

In [ ]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/20
582/582 [==============================] - 104s 178ms/step - loss: 0.0548 - accuracy: 0.9813 - val_loss: 0.3195 - val_accuracy: 0.9055
Epoch 2/20
582/582 [==============================] - 104s 179ms/step - loss: 0.0368 - accuracy: 0.9869 - val_loss: 0.2883 - val_accuracy: 0.9166
Epoch 3/20
582/582 [==============================] - 103s 176ms/step - loss: 0.0295 - accuracy: 0.9901 - val_loss: 0.2996 - val_accuracy: 0.9141
Epoch 4/20
582/582 [==============================] - 103s 178ms/step - loss: 0.0482 - accuracy: 0.9830 - val_loss: 0.2958 - val_accuracy: 0.9153
Epoch 5/20
582/582 [==============================] - 103s 177ms/step - loss: 0.0351 - accuracy: 0.9889 - val_loss: 0.3020 - val_accuracy: 0.9160
Epoch 6/20
582/582 [==============================] - 104s 179ms/step - loss: 0.0256 - accuracy: 0.9910 - val_loss: 0.3180 - val_accuracy: 0.9115
Epoch 7/20
582/582 [==============================] - 97s 167ms/step - loss: 0.0272 - accuracy: 0.9912 - val_loss: 0.3067 - 

In [ ]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/20
582/582 [==============================] - 105s 180ms/step - loss: 0.0293 - accuracy: 0.9894 - val_loss: 0.3594 - val_accuracy: 0.9044
Epoch 2/20
582/582 [==============================] - 105s 180ms/step - loss: 0.0310 - accuracy: 0.9894 - val_loss: 0.2881 - val_accuracy: 0.9226
Epoch 3/20
582/582 [==============================] - 101s 173ms/step - loss: 0.0314 - accuracy: 0.9899 - val_loss: 0.3118 - val_accuracy: 0.9160
Epoch 4/20
582/582 [==============================] - 103s 177ms/step - loss: 0.0252 - accuracy: 0.9915 - val_loss: 0.3282 - val_accuracy: 0.9175
Epoch 5/20
582/582 [==============================] - 106s 182ms/step - loss: 0.0186 - accuracy: 0.9940 - val_loss: 0.3526 - val_accuracy: 0.9186
Epoch 6/20
582/582 [==============================] - 109s 187ms/step - loss: 0.0392 - accuracy: 0.9875 - val_loss: 0.3608 - val_accuracy: 0.9074
Epoch 7/20
582/582 [==============================] - 106s 182ms/step - loss: 0.0299 - accuracy: 0.9901 - val_loss: 0.3192 -

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
model.evaluate(X_valid, y_valid)


146/146 [==============================] - 11s 73ms/step - loss: 0.2881 - accuracy: 0.9226


[0.2881031036376953, 0.9226471781730652]

In [ ]:
model.evaluate(X_test, y_test)

182/182 [==============================] - 13s 73ms/step - loss: 0.3231 - accuracy: 0.9225


[0.32306811213493347, 0.9224819540977478]

In [ ]:
!ls

healthy  model.h5  parkinson  sample_data


In [ ]:
from google.colab import files
files.download('model.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
y_test.shape

(5818,)

In [ ]:
y_train[:10]

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0])